In [0]:
rdd = sc.textFile("/FileStore/tables/Sales_Records.csv")

In [0]:
rdd.first()

Out[3]: 'Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit'

In [0]:
crdd= rdd.cache()

In [0]:
rdd1 = crdd.map(lambda x : x.split(','))

In [0]:
rdd1.take(1)

Out[7]: [['Region',
  'Country',
  'Item Type',
  'Sales Channel',
  'Order Priority',
  'Order Date',
  'Order ID',
  'Ship Date',
  'Units Sold',
  'Unit Price',
  'Unit Cost',
  'Total Revenue',
  'Total Cost',
  'Total Profit']]

In [0]:
header = rdd1.first()
rdd2 = rdd1.filter(lambda row: row != header)

In [0]:
rdd2.first()

Out[9]: ['Central America and the Caribbean',
 'Antigua and Barbuda ',
 'Baby Food',
 'Online',
 'M',
 '12/20/2013',
 '957081544',
 '1/11/2014',
 '552',
 '255.28',
 '159.42',
 '140914.56',
 '87999.84',
 '52914.72']

In [0]:
rdd3=rdd2.map(lambda x: (x[1],1))
grouped_rdd = rdd3.groupByKey()
values_rdd = grouped_rdd.mapValues(lambda values: sum(values))
values_rdd.collect()

Out[10]: [('Antigua and Barbuda ', 26),
 ('North Korea', 24),
 ('Federated States of Micronesia', 20),
 ('Ethiopia', 26),
 ('Saint Lucia', 29),
 ('Lebanon', 31),
 ('Austria', 28),
 ('Mexico', 23),
 ('Trinidad and Tobago', 19),
 ('Libya', 32),
 ('Algeria', 28),
 ('Estonia', 23),
 ('Saudi Arabia', 23),
 ('Montenegro', 28),
 ('Guatemala', 28),
 ('Australia', 34),
 ('Malawi', 24),
 ('Somalia', 36),
 ('Switzerland', 28),
 ('Laos', 34),
 ('Angola', 16),
 ('Mauritania', 29),
 ('Finland', 23),
 ('Belgium', 26),
 ('Kiribati', 25),
 ('Uzbekistan', 27),
 ('South Korea', 36),
 ('Nigeria', 23),
 ('South Africa', 23),
 ('Netherlands', 24),
 ('Solomon Islands', 31),
 ('Iran', 29),
 ('Equatorial Guinea', 33),
 ('Iraq', 29),
 ('Mauritius ', 38),
 ('Eritrea', 19),
 ('Ukraine', 33),
 ('Myanmar', 33),
 ('Latvia', 27),
 ('Portugal', 25),
 ('Barbados', 26),
 ('Poland', 28),
 ('Zambia', 31),
 ('Slovenia', 27),
 ('Bhutan', 30),
 ('Cyprus', 30),
 ('Monaco', 13),
 ('Gabon', 22),
 ('Norway', 30),
 ('Thailand', 2

In [0]:
final_rdd = values_rdd.repartition(1)
final_rdd.getNumPartitions()

Out[11]: 1

In [0]:
def toCSVLine(data):
   return ','.join(str(d) for d in data)

lines = final_rdd.map(toCSVLine)
lines.saveAsTextFile('/FileStore/tables/output1.csv')

In [0]:
lines.take(3)

Out[18]: ['Antigua and Barbuda ,26',
 'North Korea,24',
 'Federated States of Micronesia,20']

In [0]:
lines.map(lambda x : x.split(',')).take(2)

Out[19]: [['Antigua and Barbuda ', '26'], ['North Korea', '24']]

In [0]:

rdd4=rdd2.map(lambda x: (x[0],int(x[8])))
grouped_rdd = rdd4.groupByKey()
values_rdd = grouped_rdd.mapValues(lambda values: sum(values))
values_rdd.collect()

Out[22]: [('Asia', 3620036),
 ('Middle East and North Africa', 3013431),
 ('Australia and Oceania', 2111786),
 ('Central America and the Caribbean', 2698776),
 ('Europe', 6582322),
 ('Sub-Saharan Africa', 6642380),
 ('North America', 484760)]

In [0]:
from datetime import datetime
rdd5= rdd2.map(lambda x : (x[6],datetime(int(x[5]))))

In [0]:
rdd5.take(2)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
<command-1168763489641233> in <cell line: 1>()
----> 1 rdd5.take(2)

/databricks/spark/python/pyspark/instrumentation_utils.py in wrapper(*args, **kwargs)
     46             start = time.perf_counter()
     47             try:
---> 48                 res = func(*args, **kwargs)
     49                 logger.log_success(
     50                     module_name, class_name, function_name, time.perf_counter() - start, signature

/databricks/spark/python/pyspark/rdd.py in take(self, num)
   1923 
   1924             p = range(partsScanned, min(partsScanned + numPartsToTry, totalParts))
-> 1925             res = self.context.runJob(self, takeUpToNumLeft, p)
   1926 
   1927             items += res

/databricks/spark/python/pyspark/context.py in runJob(self, rdd, partitionFunc, partitions, allowLocal)
   1643             finally:
   1644  

In [0]:
rdd5 = rdd2.map(lambda row: row[:5] + [datetime.strptime(row[5],'%m/%d/%Y')] + row[6:])

In [0]:
rdd5.take(2)

Out[39]: [['Central America and the Caribbean',
  'Antigua and Barbuda ',
  'Baby Food',
  'Online',
  'M',
  datetime.datetime(2013, 12, 20, 0, 0),
  '957081544',
  '1/11/2014',
  '552',
  '255.28',
  '159.42',
  '140914.56',
  '87999.84',
  '52914.72'],
 ['Central America and the Caribbean',
  'Panama',
  'Snacks',
  'Offline',
  'C',
  datetime.datetime(2010, 7, 5, 0, 0),
  '301644504',
  '7/26/2010',
  '2167',
  '152.58',
  '97.44',
  '330640.86',
  '211152.48',
  '119488.38']]

In [0]:
rdd5.sortBy(lambda row : row[5],ascending=False).map(lambda row : row[:5] + [row[5].strftime('%-m/%-d/%Y')] +row[6:]).take(10)

Out[40]: [['Asia',
  'Bhutan',
  'Cereal',
  'Offline',
  'M',
  '7/28/2017',
  '223854434',
  '8/25/2017',
  '2356',
  '205.70',
  '117.11',
  '484629.20',
  '275911.16',
  '208718.04'],
 ['Sub-Saharan Africa',
  'Senegal',
  'Cosmetics',
  'Online',
  'C',
  '7/26/2017',
  '537970721',
  '8/18/2017',
  '6346',
  '437.20',
  '263.33',
  '2774471.20',
  '1671092.18',
  '1103379.02'],
 ['Middle East and North Africa',
  'United Arab Emirates',
  'Household',
  'Online',
  'C',
  '7/26/2017',
  '419542396',
  '8/8/2017',
  '773',
  '668.27',
  '502.54',
  '516572.71',
  '388463.42',
  '128109.29'],
 ['Australia and Oceania',
  'Australia',
  'Beverages',
  'Online',
  'L',
  '7/26/2017',
  '631485402',
  '8/12/2017',
  '9418',
  '47.45',
  '31.79',
  '446884.10',
  '299398.22',
  '147485.88'],
 ['Sub-Saharan Africa',
  "Cote d'Ivoire",
  'Vegetables',
  'Online',
  'H',
  '7/24/2017',
  '588388097',
  '8/25/2017',
  '5968',
  '154.06',
  '90.93',
  '919430.08',
  '542670.24',
  '376759.8

In [0]:
rdd6 = rdd2.map(lambda x : (x[2],x[2].count('a')))

In [0]:
rdd6.take(2)

Out[49]: [('Baby Food', 1), ('Snacks', 1)]

In [0]:
rdd6.filter(lambda x : x[1]>1).distinct().take(4)

Out[53]: [('Personal Care', 2)]

In [0]:
data = rdd2.map(lambda row: (row[0], row[1], int(row[8])))

max_unit_sold = data.groupBy(lambda row: (row[0], row[1])).mapValues(lambda rows: max(rows, key=lambda x: x[2]))

result = max_unit_sold.groupBy(lambda row: row[1][0]).mapValues(lambda rows: max(rows, key=lambda x: x[1][2]))

In [0]:
result.map(lambda row: row[1][1]).collect()

Out[61]: [('Asia', 'Thailand', 9991),
 ('Australia and Oceania', 'Samoa ', 9990),
 ('Middle East and North Africa', 'Libya', 9881),
 ('Central America and the Caribbean', 'Dominica', 9996),
 ('Europe', 'United Kingdom', 9999),
 ('Sub-Saharan Africa', 'Equatorial Guinea', 9984),
 ('North America', 'Greenland', 9898)]

In [0]:
rdd2.map(lambda row: (row[2],row[9],float(row[10]))).sortBy(lambda row: row[2] ,ascending=True).distinct().take(10)

Out[67]: [('Beverages', '47.45', 31.79),
 ('Clothes', '109.28', 35.84),
 ('Personal Care', '81.73', 56.67),
 ('Snacks', '152.58', 97.44),
 ('Baby Food', '255.28', 159.42),
 ('Meat', '421.89', 364.69),
 ('Fruits', '9.33', 6.92),
 ('Vegetables', '154.06', 90.93),
 ('Cereal', '205.70', 117.11),
 ('Cosmetics', '437.20', 263.33)]

In [0]:
rdd5.map(lambda row : (row[5].year,int(row[8]))).groupByKey().mapValues(lambda row: sum(row)).collect()

Out[80]: [(2010, 3130137),
 (2016, 3280818),
 (2014, 3214899),
 (2012, 3485045),
 (2013, 3358584),
 (2011, 3352394),
 (2015, 3506548),
 (2017, 1825066)]

In [0]:
rdd2.map(lambda row: (row[2],1)).groupByKey().mapValues(lambda row: sum(row)).collect()

Out[81]: [('Baby Food', 445),
 ('Snacks', 398),
 ('Cereal', 385),
 ('Clothes', 386),
 ('Cosmetics', 424),
 ('Fruits', 447),
 ('Beverages', 447),
 ('Personal Care', 415),
 ('Office Supplies', 420),
 ('Meat', 399),
 ('Vegetables', 410),
 ('Household', 424)]

In [0]:
rdd2.map(lambda row: (row[1],int(row[8]))).groupByKey().mapValues(lambda row: sum(row)).max()

Out[85]: ('Zimbabwe', 93906)